# Analysis of Tinnitus

This notebook applies the `Get_Job_Script.ipynb` to automatically generate the sbatch scripts to run in Yale's cluster. The end result is to apply [various LMM workflows](https://github.com/statgenetics/UKBB_GWAS_dev/tree/master/workflow) to perform association analysis in the tinnitus trait, do clumping analysis and extract associated regions.

## File paths on Yale cluster

- Genotype files in PLINK format:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv`
- Genotype files in bgen format:
`SAY/dbgapstg/scratch/UKBiobank/genotype_files/ukb39554_imputeddataset/`
- Summary stats for imputed variants BOLT-LMM:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data`
- Summary stats for inputed variants FastGWA:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data`
- Phenotype files:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment`
- Relationship file:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620`

## 07/01/20 analysis

On the cluster, open up this notebook using the JupyterLab server you set up via the ssh channel, then run the following cells,

## Bash variables for workflow configuration

In [1]:
# Common variables
tpl_file=../farnam.yml
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated082020removedwithdrawnindiv.bed
sampleFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample
bgenFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr{1..22}_v3.bgen`
unrelated_samples=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620
formatFile_fastgwa=~/project/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=~/project/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=~/project/UKBB_GWAS_dev/data/saige_template.yml
formatFile_regenie=~/project/UKBB_GWAS_dev/data/regenie_template.yml
# Container
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif
# LMM directories
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus_p3_082720
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/tinnitus
lmm_dir_saige=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/SAIGE_results/results_imputed_data/tinnitus
lmm_dir_regenie=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/REGENIE_results/results_imputed_data/tinnitus
lmm_sos=~/project/bioworkflows/GWAS/LMM.ipynb
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_tinnitus_p3_imp-fastgwa.sbatch
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_tinnitus_imp-bolt.sbatch
lmm_sbatch_saige=../output/$(date +"%Y-%m-%d")_tinnitus_imp-saige.sbatch
lmm_sbatch_regenie=../output/$(date +"%Y-%m-%d")_tinnitus_imp-regenie.sbatch
#Phenotype with imputed noise variables using the median analysis plan 1, 2 and 2
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200811_UKBB_Tinnitus_plan1_2_3_f4803
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment/200811_UKBB_Tinnitus_plan1_2_3_f4803
## LMM variables 
LDscoresFile=~/software/BOLT-LMM_v2.3.4/tables/LDSCORE.1000G_EUR.tab.gz
geneticMapFile=~/software/BOLT-LMM_v2.3.4/tables/genetic_map_hg19_withX.txt.gz
phenoCol="tinnitus"
covarCol="sex noise_imp music_imp"
covarMaxLevels=10
qCovarCol=age
numThreads=20
bgenMinMAF=0.001
bgenMinINFO=0.8
lmm_job_size=1
ylim=0
### Specific to FastGWA
grmFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.grm.sp
### Specific to SAIGE
bgenMinMAC=4
trait_type=binary
loco=TRUE
sampleCol=IID
### Specific to REGENIE
bsize=1000
lowmem=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/REGENIE_results/results_imputed_data/tinnitus_test
trait=bt
minMAC=5
reverse_log_p=True
# LD clumping directories
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/tinnitus
clumping_sos=~/project/bioworkflows/GWAS/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_tinnitus_imp_ldclumping.sbatch
## LD clumping variables
# For sumtastsFiles if more than one provide each path
bfile_ref=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.1210.ref_geno.bed
# Changes dependending upon which traits are analyzed
# In this case tinnitus only
sumstatsFiles=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus/*.snp_stats.gz
ld_sample_size=1210
clump_field=P
clump_p1=5e-08
clump_p2=1
clump_r2=0.2
clump_kb=2000
clump_annotate=BP
numThreads=20
clump_job_size=1
clumpFile= 
clumregionFile=
# Region extraction directories
extract_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/tinnitus
extract_sos=~/project/bioworkflows/GWAS/Region_Extraction.ipynb
extract_sbatch=../output/$(date +"%Y-%m-%d")_tinnitus_imp-region.sbatch
## Region extraction variables
region_file=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/tinnitus/*.clumped_region
geno_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/UKBB_bgenfilepath.txt
sumstats_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus/*.snp_stats.gz
extract_job_size=10

## BoltLMM job

In [8]:
lmm_args="""boltlmm
    --cwd $lmm_dir_bolt 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_bolt 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_option $container_option    
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_bolt \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-08-27_tinnitus_imp-bolt.sbatch
INFO: Workflow farnam (ID=5ac7ae71df3a1ef8) is executed successfully with 1 completed step.



## FastGWA job

In [2]:
lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --grmFile $grmFile
    --ylim $ylim
    --container_option $container_option
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-08-31_tinnitus_p3_imp-fastgwa.sbatch
INFO: Workflow farnam (ID=c3a0a3fdda2d91ab) is executed successfully with 1 completed step.



## SAIGE job

In [ ]:
lmm_args="""SAIGE
    --cwd $lmm_dir_saige 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_saige 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --bgenMinMAC $bgenMinMAC
    --trait_type $trait_type
    --loco $loco
    --sampleCol $sampleCol
    --job_size $lmm_job_size
    --ylim $ylim
    --container_option $container_option
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_saige \
    --args "$lmm_args"

## REGENIE job

In [2]:
maf_filter=0.001
lmm_dir_regenie=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/REGENIE_results/results_imputed_data/tinnitus_test
phenocol="tinnitus"

lmm_args="""regenie
    --cwd $lmm_dir_regenie 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_regenie 
    --phenoCol $phenoCol 
    --covarCol $covarCol  
    --qCovarCol $qCovarCol
    --bsize $bsize
    --lowmem_prefix $lowmem_prefix
    --trait $trait
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO
    --maf_filter $maf_filter
    --minMAC $minMAC
    --job_size $lmm_job_size
    --ylim $ylim
    --reverse_log_p $reverse_log_p
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_regenie \
    --args "$lmm_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-09-16_tinnitus_imp-regenie.sbatch
INFO: Workflow farnam (ID=e4321fa8c9c1e23f) is executed successfully with 1 completed step.



## LD clumping job

In [ ]:
clumping_args="""default 
    --cwd $clumping_dir 
    --bfile $bfile
    --bfile_ref $bfile_ref 
    --bgenFile $bgenFile
    --sampleFile $sampleFile 
    --sumstatsFiles $sumstatsFiles 
    --unrelated_samples $unrelated_samples 
    --ld_sample_size $ld_sample_size 
    --clump_field $clump_field
    --clump_p1 $clump_p1 
    --clump_p2 $clump_p2 
    --clump_r2 $clump_r2 
    --clump_kb $clump_kb 
    --clump_annotate $clump_annotate 
    --numThreads $numThreads 
    --job_size $clump_job_size
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $clumping_sos \
    --to-script $clumping_sbatch \
    --args "$clumping_args"

## Region extraction job

In [ ]:
extract_args="""default
    --cwd $extract_dir
    --region-file $region_file
    --pheno-path $phenoFile
    --geno-path $geno_path
    --bgen-sample-path $sampleFile
    --sumstats-path $sumstats_path
    --format-config-path $formatFile_fastgwa
    --unrelated-samples $unrelated_samples
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $extract_sos \
    --to-script $extract_sbatch \
    --args "$extract_args"

## Post-GWAS gene annotation

In this case we use the SnpGeneSets program to annotate variants to the closest gene (2000bp upstream/downstream)

In [1]:
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus_082720
postgwa_sbatch=../output/$(date +"%Y-%m-%d")_tinnitus_p1_postgwa.sbatch
sumstatsFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus_082720/200811_UKBB_Tinnitus_plan1_2_3_f4803_tinnitus.fastGWA.snp_stats_original_columns.gz
tpl_file=../farnam.yml
postgwa_sos=~/project/UKBB_GWAS_dev/workflow/snptogene.ipynb
job_size=1

postgwa_args="""default
    --cwd $lmm_dir_fastgwa
    --sumstatsFile $sumstatsFile
    --job_size $job_size
"""

sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $postgwa_sos \
    --to-script $postgwa_sbatch \
    --args "$postgwa_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-01_tinnitus_p1_postgwa.sbatch
INFO: Workflow farnam (ID=8395f44382ad5343) is executed successfully with 1 completed step.


## Hudson plots

In [ ]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots
hudson_sbatch=../output/$(date +"%Y-%m-%d")_tinnitus_p1_vs_tinnitus_p2_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus_082720/200811_UKBB_Tinnitus_plan1_2_3_f4803_tinnitus.fastGWA.snp_stats.gz
sumstats_2=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus_p2_082720/200811_UKBB_Tinnitus_plan1_2_3_f4803_tinnitus_p2.fastGWA.snp_stats.gz
toptitle="Tinnitus plan1"
bottomtitle="Tinnitus plan2"
highlight_p_top=5e-08
highlight_p_bottom=5e-08
pval_filter=5e-08
job_size=1

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
"""
sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"

In [3]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots
hudson_sbatch=../output/$(date +"%Y-%m-%d")_tinnitus_p1_vs_tinnitus_p3_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus_082720/200811_UKBB_Tinnitus_plan1_2_3_f4803_tinnitus.fastGWA.snp_stats.gz
sumstats_2=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/tinnitus_p3_082720/200811_UKBB_Tinnitus_plan1_2_3_f4803_tinnitus_p3.fastGWA.snp_stats.gz
toptitle="Tinnitus plan1"
bottomtitle="Tinnitus plan3"
highlight_p_top=5e-08
highlight_p_bottom=5e-08
pval_filter=5e-08
job_size=1

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
"""
sos run ~/project/bioworkflows/GWAS/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"

INFO: Running farnam: Configuration for Yale `farnam` cluster
INFO: farnam is completed.
INFO: farnam output:   ../output/2020-10-14_tinnitus_p1_vs_tinnitus_p3_hudson.sbatch
INFO: Workflow farnam (ID=dca7658f8a7edd2f) is executed successfully with 1 completed step.

